In [78]:
from pathlib import Path

acme_data = (
    "MBI|FNAME|LNAME|DOB|EMAIL|PHONE\n"
    "1234567890A|John|Doe|03/15/1955|JOHN.DOE@EMAIL.COM|5551234567\n"
    "9876543210B|Jane|Smith|07/22/1948|jane.smith@email.com|5559876543\n"
)

Path("/content/acme.txt").write_text(acme_data)


160

In [79]:
from pathlib import Path

bettercare_data = (
    "subscriber_id,first_name,last_name,date_of_birth,email,phone\n"
    "BC-001,Alice,Johnson,1965-08-10,alice.j@test.com,555-222-3333\n"
    "BC-002,Charlie,Brown,1972-03-25,charlie.b@test.com,5554445555\n"
)

Path("/content/bettercare.csv").write_text(bettercare_data)


185

In [80]:
PARTNER_CONFIG = dict(
    acme=dict(
        file_path="/content/acme.txt",
        delimiter="|",
        partner_code="ACME",
        column_mapping=dict(
            MBI="external_id",
            FNAME="first_name",
            LNAME="last_name",
            DOB="dob",
            EMAIL="email",
            PHONE="phone"
        )
    ),
    bettercare=dict(
        file_path="/content/bettercare.csv",
        delimiter=",",
        partner_code="BETTERCARE",
        column_mapping=dict(
            subscriber_id="external_id",
            first_name="first_name",
            last_name="last_name",
            date_of_birth="dob",
            email="email",
            phone="phone"
        )
    )
)


In [81]:
def format_dob(dob):
    if pd.isna(dob):
        return None

    parsed_date = pd.to_datetime(dob, errors="coerce")

    if pd.isna(parsed_date):
        return None

    return parsed_date.strftime("%Y-%m-%d")


In [82]:
def format_phone(phone):
    if phone is None or pd.isna(phone):
        return None

    digits = "".join(filter(str.isdigit, str(phone)))

    if len(digits) != 10:
        return None

    return f"{digits[:3]}-{digits[3:6]}-{digits[6:]}"


In [83]:
def validate_records(df):
    required_cols = ["external_id", "dob", "phone"]

    good_records = df.dropna(subset=required_cols)
    bad_records = df.loc[df.index.difference(good_records.index)]

    return good_records, bad_records


In [84]:
def ingest_partner(partner_name, config):
    cfg = config[partner_name]

    try:
        raw_df = pd.read_csv(cfg["file_path"], sep=cfg["delimiter"])
    except Exception as e:
        print(f"Failed to read file for {partner_name}: {e}")
        return pd.DataFrame()

    standardized_df = (
        raw_df
        .rename(columns=cfg["column_mapping"])
        .loc[:, cfg["column_mapping"].values()]
        .assign(
            first_name=lambda x: x["first_name"].str.title(),
            last_name=lambda x: x["last_name"].str.title(),
            email=lambda x: x["email"].str.lower(),
            dob=lambda x: x["dob"].apply(format_dob),
            phone=lambda x: x["phone"].apply(format_phone),
            partner_code=cfg["partner_code"]
        )
    )

    valid_df, invalid_df = validate_records(standardized_df)

    print(
        f"{partner_name.upper()} | "
        f"Total: {len(standardized_df)} | "
        f"Valid: {len(valid_df)} | "
        f"Rejected: {len(invalid_df)}"
    )

    return valid_df


In [85]:
import pandas as pd
final_df = pd.concat(
    [ingest_partner(p, PARTNER_CONFIG) for p in PARTNER_CONFIG],
    ignore_index=True
)


ACME | Total: 2 | Valid: 2 | Rejected: 0
BETTERCARE | Total: 2 | Valid: 2 | Rejected: 0


In [86]:
final_df

,external_id,first_name,last_name,dob,email,phone,partner_code
0,1234567890A,John,Doe,1955-03-15,john.doe@email.com,555-123-4567,ACME
1,9876543210B,Jane,Smith,1948-07-22,jane.smith@email.com,555-987-6543,ACME
2,BC-001,Alice,Johnson,1965-08-10,alice.j@test.com,555-222-3333,BETTERCARE
3,BC-002,Charlie,Brown,1972-03-25,charlie.b@test.com,555-444-5555,BETTERCARE


In [87]:
from pathlib import Path

output_path = Path("/content") / "final_eligibility_output.csv"
final_df.to_csv(output_path, index=False)


In [88]:
def validate_records(df):
    def is_invalid(row):
        return (
            pd.isna(row["external_id"]) or
            pd.isna(row["dob"]) or
            pd.isna(row["phone"])
        )

    invalid_rows = df.apply(is_invalid, axis=1)

    bad_records = df[invalid_rows]
    good_records = df[~invalid_rows]

    return good_records, bad_records


In [89]:
summary = (
    final_df
    .groupby("partner_code", as_index=True)
    .agg({
        "external_id": ["count", "nunique"]
    })
)

summary.columns = ["total_records", "unique_members"]
summary


,total_records,unique_members
partner_code,,
ACME,2,2
BETTERCARE,2,2


In [90]:
from pathlib import Path

excel_path = Path("/content/final_eligibility_output.xlsx")
final_df.to_excel(excel_path, index=False)

print(f"Excel file saved at: {excel_path.resolve()}")


Excel file saved at: /content/final_eligibility_output.xlsx
